In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = True

class DogsVSCats():
    img_size = 50
    cats = 'training_set/cats'
    dogs = 'training_set/dogs'
    labels = {cats: 0, dogs: 1}
    
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        for label in self.labels:
            print (label)
            
            for f in tqdm(os.listdir(label)):
                
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    self.training_data.append([np.array(img), np.eye(2)[self.labels[label]]])
                    
                    #print (self.cats)
                    
                    if (label == self.cats):
                        self.catcount += 1
                    elif (label == self.dogs):
                        self.dogcount += 1

                except Exception as e:
                    pass
                
        np.random.shuffle(self.training_data)
        np.save('train_data.npy', self.training_data)
        print ('cats', self.catcount)
        print ('dogs', self.dogcount)
        
if REBUILD_DATA:
    dogvcats = DogsVSCats()
    dogvcats.make_training_data()

training_set/cats


100%|███████████████████████████████████████████████| 4001/4001 [00:02<00:00, 1706.08it/s]


training_set/dogs


100%|███████████████████████████████████████████████| 4006/4006 [00:02<00:00, 1601.38it/s]
/home/dark-knight/.local/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


cats 4000
dogs 4005


In [2]:
training_data = np.load("train_data.npy", allow_pickle=True)
print(len(training_data))

8005


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
    
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        #print (x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2] 
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net()
    

In [4]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

val_pct = 0.1
val_size = int(len(X)*val_pct)
print (val_size)

800


/tmp/ipykernel_6408/95528811.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)


In [5]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[:-val_size]
test_y = y[:-val_size]

print(len(train_X))
print(len(test_X))

7205
7205


correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))
        
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1
    
print ('acc', round(correct/total, 3))
        
    

In [6]:
torch.cuda.is_available()

True

In [7]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print ('gpu')
else:
    device = torch.device('cpu')
    print ('cpu')

gpu


In [9]:
torch.cuda.device_count()

1

In [10]:
#net.to(device)
# Net(
#   (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
#   (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
#   (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
#   (fc1): Linear(in_features=512, out_features=512, bias=True)
#   (fc2): Linear(in_features=512, out_features=2, bias=True)
# )

import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [11]:
net = Net().to(device)
print (net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [12]:
batch_size = 100
epochs = 3

def train(net):
    optimizer = optim.Adam(net.parameters(), lr = 0.001)
    loss_function = nn.MSELoss()
    for e in range(epochs):
        for i in tqdm(range(0, len(train_X), batch_size)):
            #print (i, i+batch_size)
            batch_X = train_X[i:i+batch_size].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+batch_size]
            
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print (loss)
    
train(net)

100%|█████████████████████████████████████████████████████| 73/73 [00:02<00:00, 27.26it/s]


tensor(0.2688, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████| 73/73 [00:00<00:00, 188.92it/s]


tensor(0.2107, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████| 73/73 [00:00<00:00, 187.12it/s]

tensor(0.2249, device='cuda:0', grad_fn=<MseLossBackward0>)


In [13]:
#train(net)

In [14]:
def test(net):
    correct = 0
    total = 0

    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print ('acc', round(correct/total, 3))

test(net)

100%|███████████████████████████████████████████████| 7205/7205 [00:02<00:00, 3598.54it/s]

acc 0.599


In [18]:
def fwd_pass(X, y, train=False):
    if train:
        net.zero_grad()
    
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    
    return acc, loss

In [19]:

def test(size=32):
    
    random_start = np.random.randint(len(test_X)-size)
    X, y = test_X[random_start:random_start+size], test_y[random_start:random_start+size]
    
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(X.view(-1, 1, 50, 50).to(device), y.to(device))
    
    return val_acc, val_loss

val_acc, val_loss = test(size=32)
print(val_acc, val_loss)
    

0.625 tensor(0.2215, device='cuda:0')


In [ ]:
torch.cuda.is_available()

In [20]:
import time

MODEL_NAME = f"model-{int(time.time())}"  # every time models runs it has a new name with time.

net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

print (MODEL_NAME)

def train():
    batch_size = 100
    epochs = 8
    with open("model.log", "a") as f:
        for e in range(epochs):
            for i in tqdm(range(0, len(train_X), batch_size)):
                batch_X = train_X[i:i+batch_size].view(-1, 1, 50, 50).to(device)
                batch_y = train_y[i:i+batch_size].to(device)
                
                acc, loss = fwd_pass(batch_X, batch_y, train=True)
                if (i % 50 ==0):
                    val_acc, val_loss = test(size = 100)
                    f.write (f"{MODEL_NAME},{round(time.time(),3)},{round(float(acc),2)}, {round(float(loss),4)}, {round(float(val_acc),2)}, {round(float(val_loss),4)}\n")
                    
train()

model-1685790665


100%|█████████████████████████████████████████████████████| 73/73 [00:00<00:00, 92.11it/s]
